In [216]:
import time

class Cell:
    def __init__(self, initial_value = False):
        if not initial_value:
            self.domain = set([1,2,3,4,5,6,7,8,9])
        else:
            assert initial_value in range(1,10)
            self.domain = set([initial_value])
            
    def remove(self, item):
        if item in self.domain:
            self.domain.remove(item)
            return True
        else:
            return False
        
    def get_known_cell_value(self):
        assert len(self.domain) == 1
        for elem in self.domain:
            return elem
    
    def is_known(self):
        return len(self.domain) == 1

empty_cell = "."
class Sudoku:
    board = [["empty" for _ in range(9)] for _ in range(9)]
    known_indices = set([])
    unknown_indices = set([])
    def __init__(self, list_board):
        #assume '.' denodes empty cells, '4' would be a valued cell
        for row in range(9):
            for col in range(9):
                list_item = list_board[row][col]
                if list_item == empty_cell:
                    self.board[row][col] = Cell()
                    self.unknown_indices.add((row, col))
                else:
                    self.board[row][col] = Cell(int(list_item))
                    self.known_indices.add((row, col))
                    
    def is_solved(self):
        return len(self.unknown_indices) == 0
    
    def has_valid_domains(self):
        for row in self.board:
            for cell in row:
                if len(cell.domain) == 0:
                    return False
        return True
    def get_cell(self, row, col):
        return self.board[row][col]
                    
    def print_board(self):
        for row in self.board:
            buff = []
            for cell in row:
                if cell.is_known():
                    buff.append(cell.get_known_cell_value())
                else:
                    buff.append(empty_cell)
            print(*buff)
    
    def return_board(self):
        board_list = []
        for row in self.board:
            buff = []
            for cell in row:
                if cell.is_known():
                    buff.append(str(cell.get_known_cell_value()))
                else:
                    buff.append(empty_cell)
            board_list.append(buff)
        return board_list
    
    def row_indices(self, row, col):
        return set([(row_i, col) for row_i in range(9) if row_i != row])
    
    def col_indices(self, row, col):
        return set([(row, col_i) for col_i in range(9) if col_i != col])
        
    def sub_block_indices(self, row, col):
        sub_row = row // 3
        sub_col = col // 3
        to_return = set([(3*sub_row + row_i, 3*sub_col + col_i) for row_i in range(3) for col_i in range(3)])
        to_return.remove((row, col))
        return to_return
    
    def get_constrained_indices(self, row, col):
        return self.row_indices(row, col).union(self.col_indices(row, col)).union(self.sub_block_indices(row, col))
    
    def arc_consistency_outwards(self):
        '''ex: if a cell is 2, 2 is removed from domain of cells in row, col and sub_block'''
        removed_count = 0
        now_known_indices = set([])
        for known_index in self.known_indices:
            cell = self.get_cell(*known_index)
            cell_value = cell.get_known_cell_value()
            for constrained_index in self.unknown_indices.intersection(self.get_constrained_indices(*known_index)):
                constrained_cell = self.get_cell(*constrained_index)
                if constrained_cell.remove(cell_value) == True:
                    removed_count += 1
                    if constrained_cell.is_known():
                        self.unknown_indices.remove(constrained_index)
                        now_known_indices.add(constrained_index)
                        
        self.known_indices = self.known_indices.union(now_known_indices)
        self.unknown_indices.difference_update(now_known_indices)
        return removed_count
    
    def all_different_constraint(self):
        '''ex: if unknown_cell is the only cell in its row that can be a 2, unknown cell must be a 2'''
        now_known_indices = set([])
        for unknown_index in self.unknown_indices:
            unknown_cell = self.get_cell(*unknown_index)
            
            row_domain = set([])
            for row_index in self.row_indices(*unknown_index):
                row_domain = row_domain.union(self.get_cell(*row_index).domain)
            if len(unknown_cell.domain.difference(row_domain)) == 1:
                unknown_cell.domain = unknown_cell.domain.difference(row_domain)
                now_known_indices.add(unknown_index)
                continue
                
            col_domain = set([])
            for col_index in self.col_indices(*unknown_index):
                col_domain = col_domain.union(self.get_cell(*col_index).domain)
            if len(unknown_cell.domain.difference(col_domain)) == 1:
                unknown_cell.domain = unknown_cell.domain.difference(col_domain)
                now_known_indices.add(unknown_index)
                continue
                
            sub_block_domain = set([])
            for sub_index in self.sub_block_indices(*unknown_index):
                sub_block_domain = sub_block_domain.union(self.get_cell(*sub_index).domain)
            if len(unknown_cell.domain.difference(sub_block_domain)) == 1:
                unknown_cell.domain = unknown_cell.domain.difference(sub_block_domain)
                now_known_indices.add(unknown_index)
                continue
            
        self.known_indices = self.known_indices.union(now_known_indices)
        self.unknown_indices.difference_update(now_known_indices)
        
    def solve_easy(self, debug=False):
        '''runs all_different_constraints and arc_consistency_outwards until no new cell values are found'''
        start = time.time()
        prev_known_indices = -1
        while len(self.known_indices) != prev_known_indices:
            prev_known_indices = len(self.known_indices)
            if debug == True:
                print("known:", len(self.known_indices))
                self.print_board()
            self.arc_consistency_outwards()
            self.all_different_constraint()
        print("completion time (s)", time.time() - start)
        
    def solve(self):
        stack = [self.return_board()]
        while stack:
            print("Loop")
            board_as_list = stack.pop()
            S = Sudoku(board_as_list)
            S.solve_easy(debug = False)
            if S.is_solved():
                self = S
                break
            if not S.has_valid_domains():
                continue
            min_remaining_vals_index = min(S.unknown_indices, key=lambda index : len(S.get_cell(*index).domain))
            mrv_row, mrv_col = min_remaining_vals_index
            for guess in S.get_cell(*min_remaining_vals_index).domain:
                new_test = S.return_board()
                new_test[mrv_row][mrv_col] = guess
                stack.append(new_test)
        

In [217]:
newspaper = [
    ".8.....1.",
    "..3.9.7..",
    "1..8.7..6",
    ".2.6.3.5.",
    "....8....",
    ".6.1.2.4.",
    "3..9.8..2",
    "..2.6.4..",
    ".7.....8.",]
#http://sudokukid.com/play/expert/327
possibly_not_unique = [
    "7........",
    ".......12",
    "..3.4....",
    "...1.....",
    "..6...4..",
    ".1.2.7...",
    "...8.....",
    "..4.3.5..",
    ".7....9..",
]

newspaper = [list(string) for string in newspaper]
possibly_not_unique = [list(string) for string in possibly_not_unique]
test_input = [["5","3",".",".","7",".",".",".","."],["6",".",".","1","9","5",".",".","."],[".","9","8",".",".",".",".","6","."],["8",".",".",".","6",".",".",".","3"],["4",".",".","8",".","3",".",".","1"],["7",".",".",".","2",".",".",".","6"],[".","6",".",".",".",".","2","8","."],[".",".",".","4","1","9",".",".","5"],[".",".",".",".","8",".",".","7","9"]]
test_input2 = [[".",".","9","7","4","8",".",".","."],["7",".",".",".",".",".",".",".","."],[".","2",".","1",".","9",".",".","."],[".",".","7",".",".",".","2","4","."],[".","6","4",".","1",".","5","9","."],[".","9","8",".",".",".","3",".","."],[".",".",".","8",".","3",".","2","."],[".",".",".",".",".",".",".",".","6"],[".",".",".","2","7","5","9",".","."]]

In [219]:
S = Sudoku(possibly_not_unique)
# S.solve_easy(debug=True)
S.solve()
S.return_board()

Loop
completion time (s) 0.015474557876586914
Loop
completion time (s) 0.010224342346191406
Loop
completion time (s) 0.006248950958251953
Loop
completion time (s) 0.00563359260559082
Loop
completion time (s) 0.003034830093383789
Loop
completion time (s) 0.005629301071166992
Loop
completion time (s) 0.010572195053100586
Loop
completion time (s) 0.010133028030395508
Loop
completion time (s) 0.00729680061340332
Loop
completion time (s) 0.006583213806152344
Loop
completion time (s) 0.0065402984619140625


[['7', '5', '1', '6', '2', '8', '3', '9', '4'],
 ['6', '4', '8', '3', '5', '9', '7', '1', '2'],
 ['2', '9', '3', '7', '4', '1', '8', '5', '6'],
 ['9', '3', '7', '1', '6', '4', '2', '8', '5'],
 ['8', '2', '6', '5', '9', '3', '4', '7', '1'],
 ['4', '1', '5', '2', '8', '7', '6', '3', '9'],
 ['5', '6', '9', '8', '7', '2', '1', '4', '3'],
 ['1', '8', '4', '9', '3', '6', '5', '2', '7'],
 ['3', '7', '2', '4', '1', '5', '9', '6', '8']]

In [166]:
sorted_indices = [cell_index for cell_index in S.unknown_indices]
sorted_indices.sort()
for sorted_index in sorted_indices:
    cell = S.get_cell(*sorted_index)
    print(sorted_index, cell.domain)

(0, 2) {9}
(0, 3) {7}
(0, 4) {4}
(0, 5) {8}
(1, 0) {7}
(2, 1) {2}
(2, 3) {1}
(2, 5) {9}
(3, 2) {7}
(3, 6) {2}
(3, 7) {4}
(4, 1) {6}
(4, 2) {4}
(4, 4) {1}
(4, 6) {5}
(4, 7) {9}
(5, 1) {9}
(5, 2) {8}
(5, 6) {3}
(6, 3) {8}
(6, 5) {3}
(6, 7) {2}
(7, 8) {6}
(8, 3) {2}
(8, 4) {7}
(8, 5) {5}
(8, 6) {9}


In [81]:
min(S.unknown_indices, key=lambda index : len(S.get_cell(*index).domain))

(3, 0)